# Flowsheet initialization
This flowsheet generated multiple initialization instances at different R102 temperatures for multistart optimization

### ROK model = M5

In [ ]:
# simulation case specifications
model_code = 5
case_name = 'Bakken'

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                    define_models, \
                                    define_arcs, \
                                    set_unit_model_variables, \
                                    initialize_flowsheet, \
                                    set_scaling_factors,\
                                    update_model_after_initialization,\
                                    vapor_only_to_vapor_liquid_reformulate,\
                                    H106_inlet_vapor_reformulate, \
                                    replace_heater_heat_duty_constraint_with_bounds, \
                                    update_model_for_optimization, \
                                    unfix_DOFs_pre_optimization, \
                                    fix_DOFs_post_optimization

In [ ]:
m = create_flowsheet(model_code)

In [ ]:
M_catalyst = 1167.003367 # kg

## Define equipment and connections

In [ ]:
# define unit models
define_models(m, catalyst_mass = M_catalyst)
#define connections
define_arcs(m)

## Define inlet compositions

In [ ]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_composition_dict = {}

for col in inlet_df.columns:
    if col == case_name:
        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)

inlet_flow_rate = 481.3888889

inlet_composition_dict

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188}

inlet_composition_dict

## Define constraints and set-points for equipment

In [ ]:
set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)

## Scale model components

In [ ]:
if model_code == 2 or model_code == 3:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
elif model_code == 4 or model_code == 5:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
else:
    pass

## Initialized Flowsheet read-in

In [ ]:
ms.from_json(m, fname="./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format(case_name, model_code))

## Add post-initialization constraints

In [ ]:
update_model_after_initialization(m)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)

In [ ]:
ms.from_json(m, fname="./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))

In [ ]:
replace_heater_heat_duty_constraint_with_bounds(m)

# Flowsheet convergence

## Set _R102 and solve flowsheet

In [ ]:
# T_R102_list = [523.0,533.0,543.0,553.0,563.0,573.0,583.0,593.0,603.0,613.0,623.0]
T_R102_list = [523.0]

In [ ]:
DOF_initial = degrees_of_freedom(m)
print("The final DOF of initialized flowsheet is {0}".format(DOF_initial))

In [ ]:
for T_R102 in T_R102_list:
    print("***********************  T_R102 = {}  ***********************".format(T_R102))
    m.fs.H103.outlet.temperature.unfix()
    m.fs.H103.outlet.temperature.fix(T_R102)
    
    for i in m.fs.R102.control_volume.length_domain:
        m.fs.R102.control_volume.properties[0.0,i].temperature.unfix()
        if i != 0:
            m.fs.R102.control_volume.properties[0.0,i].temperature.fix(T_R102)
    
    # Unfix H105 DoFs: outlet T
    m.fs.H105.outlet.temperature.unfix()

    # Unfix F101 DoF: pressure drop
    m.fs.F101.deltaP.unfix()

    # Unfix H106 DoFs: outlet T and P
    m.fs.H106.outlet.pressure.unfix()

    # Unfix F102 DoF: pressure drop
    m.fs.F102.deltaP.unfix()
    
    DOF_initial = degrees_of_freedom(m)
    print("The initial DOF is {0}".format(DOF_initial))
    solver = SolverFactory('ipopt')
    solver.options = {'tol': 1e-6,
                      'bound_push': 1e-8,
                      'max_iter':200
                     }
    # solve
    solve_status = solver.solve(m, tee=True)
    
    # Fix H105 DoFs: outlet T
    m.fs.H105.outlet.temperature.fix()

    # Fix F101 DoF: pressure drop
    m.fs.F101.deltaP.fix()

    # Fix H106 DoFs: outlet T and P
    m.fs.H106.outlet.pressure.fix()

    # Fix F102 DoF: pressure drop
    m.fs.F102.deltaP.fix()
    
    ms.to_json(m, fname="./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}_T_R102={}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3),T_R102))
    m.fs.H106.report()

In [ ]:
m.fs.H106.heat_duty.pprint()

In [ ]:
for T_R102 in T_R102_list:
    ms.from_json(m, fname="./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}_T_R102={}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3),T_R102))
    print('\n **** T_R102 = {} ****'.format(round(m.fs.R102.control_volume.properties[0,1.0].temperature(),4)))
    print('T_H105 = {}'.format(round(m.fs.H105.outlet.temperature[0](),4)))
    print('dP_F101 = {}'.format(round(m.fs.F101.deltaP[0](),4)))
    print('T_H106 = {}'.format(round(m.fs.H106.outlet.temperature[0](),4)))
    print('P_H106 = {}'.format(round(m.fs.H106.outlet.pressure[0](),4)))
    print('dP_F102 = {}'.format(round(m.fs.F102.deltaP[0](),4)))